In [1]:
import numpy as np
import pandas as pd
import torch
from transformers import pipeline

# General EDA:

In [2]:
df = pd.read_csv('../datasets/labelled_newscatcher_dataset.csv', delimiter=';')
df.sample(10)

,topic,link,domain,published_date,title,lang
9591,HEALTH,https://www.rnz.co.nz/news/national/423623/tok...,rnz.co.nz,2020-08-15 04:16:31,Tokoroa Covid-19 cases spark unease in vulnera...,en
85752,BUSINESS,https://www.malaymail.com/news/money/2020/08/1...,malaymail.com,2020-08-09 23:23:53,US dollar tries to keep rally alive in a short...,en
31618,WORLD,https://www.bbc.co.uk/news/av/world-us-canada-...,bbc.co.uk,2020-08-06 05:19:34,Joe Biden: 'Why the hell would I take a cognit...,en
32210,NATION,https://www.modernghana.com/news/1022140/bomb-...,modernghana.com,2020-08-11 00:07:45,Bomb Explosion At La Beach Injures Four Children,en
22631,SPORTS,https://www.skynews.com.au/details/_6177589163001,skynews.com.au,2020-08-03 05:50:14,'HEARTBREAKING SHUTDOWN': Melbourne businesses...,en
29714,SPORTS,https://winnersandwhiners.com/games/mlb/8-6-20...,winnersandwhiners.com,2020-08-06 00:13:23,San Francisco Giants vs. Colorado Rockies Pred...,en
20347,WORLD,https://www.mirror.co.uk/news/world-news/saudi...,mirror.co.uk,2020-08-07 10:17:00,"Saudi ruler 'sent 50 hitman ""Tiger Squad"" to k...",en
2254,TECHNOLOGY,https://mashable.com/shopping/aug-11-consultan...,mashable.com,2020-08-11 09:00:00,This 12-hour master class shows you how to mak...,en
58420,NATION,https://www.heraldsun.com.au/subscribe/news/1/...,heraldsun.com.au,2020-08-06 07:00:00,Were missing high country campers attacked in ...,en
18702,ENTERTAINMENT,https://whatculture.com/tv/the-umbrella-academ...,whatculture.com,2020-08-15 09:18:35,The Umbrella Academy Season 2: Ranking EVERY C...,en


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108774 entries, 0 to 108773
Data columns (total 6 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   topic           108774 non-null  object
 1   link            108774 non-null  object
 2   domain          108774 non-null  object
 3   published_date  108774 non-null  object
 4   title           108774 non-null  object
 5   lang            108774 non-null  object
dtypes: object(6)
memory usage: 5.0+ MB


В данном датасете нет пропусков

In [4]:
df.topic.value_counts()

topic
TECHNOLOGY       15000
HEALTH           15000
WORLD            15000
ENTERTAINMENT    15000
BUSINESS         15000
SPORTS           15000
NATION           15000
SCIENCE           3774
Name: count, dtype: int64

In [5]:
df.describe()

,topic,link,domain,published_date,title,lang
count,108774,108774,108774,108774,108774,108774
unique,8,106130,5164,68743,103180,1
top,TECHNOLOGY,https://www.google.com/,dailymail.co.uk,2020-08-04 01:00:00,"US tops 5 million confirmed virus cases, to Eu...",en
freq,15000,19,1855,41,21,108774


Можно заметить, что все заголовки на английском языке, распределены по 8 тематикам, однако, не все заголовки уникальны

In [6]:
temp = df.topic.value_counts().reset_index()['count']
df.drop_duplicates(subset=['title'], inplace=True)
df.describe()

,topic,link,domain,published_date,title,lang
count,103180,103180,103180,103180,103180,103180
unique,8,100693,4951,66299,103180,1
top,NATION,https://www.google.com/,dailymail.co.uk,2020-08-04 01:00:00,Report: Talks Underway To Keep Shane Duffy In ...,en
freq,14777,19,1843,40,1,103180


In [7]:
temp_df = df.topic.value_counts().reset_index()
temp_df['prev_count'] = temp
temp_df

,topic,count,prev_count
0,NATION,14777,15000
1,SPORTS,14757,15000
2,ENTERTAINMENT,14722,15000
3,BUSINESS,14576,15000
4,WORLD,14255,15000
5,TECHNOLOGY,14255,15000
6,HEALTH,12608,15000
7,SCIENCE,3230,3774


In [8]:
temp_df['diff'] = temp_df['prev_count'] - temp_df['count']
temp_df.sort_values('diff', ascending=False)

,topic,count,prev_count,diff
6,HEALTH,12608,15000,2392
4,WORLD,14255,15000,745
5,TECHNOLOGY,14255,15000,745
7,SCIENCE,3230,3774,544
3,BUSINESS,14576,15000,424
2,ENTERTAINMENT,14722,15000,278
1,SPORTS,14757,15000,243
0,NATION,14777,15000,223


Как можно заметить, после удаления повторяющихся заголовков, наибольшее количество строк потеряла тематика "HEALTH"

In [9]:
df

,topic,link,domain,published_date,title,lang
0,SCIENCE,https://www.eurekalert.org/pub_releases/2020-0...,eurekalert.org,2020-08-06 13:59:45,A closer look at water-splitting's solar fuel ...,en
1,SCIENCE,https://www.pulse.ng/news/world/an-irresistibl...,pulse.ng,2020-08-12 15:14:19,"An irresistible scent makes locusts swarm, stu...",en
2,SCIENCE,https://www.express.co.uk/news/science/1322607...,express.co.uk,2020-08-13 21:01:00,Artificial intelligence warning: AI will know ...,en
3,SCIENCE,https://www.ndtv.com/world-news/glaciers-could...,ndtv.com,2020-08-03 22:18:26,Glaciers Could Have Sculpted Mars Valleys: Study,en
4,SCIENCE,https://www.thesun.ie/tech/5742187/perseid-met...,thesun.ie,2020-08-12 19:54:36,Perseid meteor shower 2020: What time and how ...,en
...,...,...,...,...,...,...
108769,NATION,https://www.vanguardngr.com/2020/08/pdp-govern...,vanguardngr.com,2020-08-08 02:40:00,PDP governors’ forum urges security agencies t...,en
108770,BUSINESS,https://www.patentlyapple.com/patently-apple/2...,patentlyapple.com,2020-08-08 01:27:12,"In Q2-20, Apple Dominated the Premium Smartpho...",en
108771,HEALTH,https://www.belfastlive.co.uk/news/health/coro...,belfastlive.co.uk,2020-08-12 17:01:00,Coronavirus Northern Ireland: Full breakdown s...,en
108772,ENTERTAINMENT,https://www.thenews.com.pk/latest/696364-paul-...,thenews.com.pk,2020-08-05 04:59:00,Paul McCartney details post-Beatles distress a...,en


In [10]:
df['published_date'] = pd.to_datetime(df.published_date)

In [11]:
df.published_date.min()

Timestamp('2012-09-16 04:44:50')

In [12]:
df.published_date.max()

Timestamp('2020-08-18 05:49:00')

Временной промежуток записей в датасете охватывает период с 16 сенятбря 2012 года до 18 августа 2020 года

In [13]:
df['year'] = df.published_date.dt.year

In [14]:
df.year.value_counts()

year
2020    103023
2019       109
2018        25
2017        16
2016         3
2012         3
2015         1
Name: count, dtype: int64

Однако, подавляющее большинство записей были сделаны в 2020 году

In [15]:
df['month'] = df.published_date.dt.month
df[df.year == 2020].month.value_counts()

month
8    102724
7        85
6        49
5        49
4        45
3        39
1        19
2        13
Name: count, dtype: int64

Подавляющее большинство заголовков принадлежит августу 2020 года

# Task specific EDA:

Предположим, что интересуюшая нас организация - это NASA

In [16]:
ORG_NAME = 'NASA'

In [17]:
df.title.str.contains(ORG_NAME).value_counts()

title
False    102676
True        504
Name: count, dtype: int64

Всего организация упомяналась 504 раза в следующих категориях:

In [18]:
df[df.title.str.contains(ORG_NAME)].topic.value_counts()

topic
SCIENCE          475
TECHNOLOGY        25
ENTERTAINMENT      2
HEALTH             1
NATION             1
Name: count, dtype: int64

Посмотрим несколько заголовков из разных категорий:

In [19]:
def show_titles(topic: str) -> pd.DataFrame:
    if not topic in df.topic.unique().tolist():
        raise ValueError(f'Provided topic `{topic}` must be in DataFrame topics')

    option_1 = df.title.str.contains(ORG_NAME)
    option_2 = df.topic == topic

    return df[option_1 & option_2].title

In [20]:
show_titles('HEALTH')


22611    NASA releases weird-looking shape of our solar...
Name: title, dtype: object

In [21]:
show_titles('NATION')


51281    Chandrayaan 2 Pragyan rover not damaged; New N...
Name: title, dtype: object

In [22]:
show_titles('TECHNOLOGY').sample(10)


30151    NASA succeeds in receiving laser beam signal f...
37955    NASA official may face criminal investigation ...
12939    Watch: teaming up with NASA for new 'Hyperspac...
16482    Watch: Beck teams up with NASA for new 'Hypers...
449      Justice Department opens probe into ex-NASA of...
18422    NASA Commander Chris Cassidy Is Making Music V...
48288    Beck Teams With NASA for New Hyperspace Visual...
12060    Beck collaborates with NASA on new ‘A.I. Explo...
21926    Beck Teams With NASA for ‘Hyperspace’ Visual A...
13614    U.S. Justice Dept opens probe into ex-NASA off...
Name: title, dtype: object

In [23]:
show_titles('SCIENCE').sample(10)


55800     Space X and NASA launch aborted 16 minutes bef...
38018     'Shallow Lightning' and 'Mushballs' Reveal Amm...
18113                         NASA Maps Beirut Blast Damage
142                   NASA Saved Its Mars Mole, Is It True?
6648      Russia to provide NASA with full information o...
90121     NASA to Rename Cosmic Objects, Replace Racial ...
54235     NASA Perseveres Through COVID-19 Pandemic – He...
66057     Roscosmos space agency invites NASA top offici...
65996     Two galaxies collide in NASA's majestic Hubble...
103023    NASA Removes 'Insensitive' Nicknames For Certa...
Name: title, dtype: object

In [24]:
df[df.title.str.contains(ORG_NAME)]

,topic,link,domain,published_date,title,lang,year,month
5,SCIENCE,https://interestingengineering.com/nasa-releas...,interestingengineering.com,2020-08-08 11:05:45,NASA Releases In-Depth Map of Beirut Explosion...,en,2020,8
6,SCIENCE,https://www.thequint.com/tech-and-auto/spacex-...,thequint.com,2020-05-28 09:09:46,"SpaceX, NASA Demo-2 Rocket Launch Set for Satu...",en,2020,5
9,SCIENCE,https://www.thehindubusinessline.com/news/scie...,thehindubusinessline.com,2020-08-14 07:43:25,NASA invites engineering students to help harv...,en,2020,8
11,SCIENCE,https://interestingengineering.com/nasa-finds-...,interestingengineering.com,2020-08-07 09:23:57,NASA Finds Ammonia-Linked 'Mushballs' and 'Sha...,en,2020,8
26,SCIENCE,https://www.clickorlando.com/news/local/2020/0...,clickorlando.com,2020-08-04 23:21:00,‘It came alive:’ NASA astronauts describe expe...,en,2020,8
...,...,...,...,...,...,...,...,...
107512,SCIENCE,https://www.haveeru.com.mv/to-start-with-at-an...,haveeru.com.mv,2020-08-06 20:30:16,To start with-at any time place helicopter is ...,en,2020,8
108092,SCIENCE,https://solarsystem.nasa.gov/news/1421/arecibo...,solarsystem.nasa.gov,2020-08-07 17:38:24,Arecibo Observatory Returns from Tropical Stor...,en,2020,8
108269,SCIENCE,https://americaspeaksink.com/2020/08/nasa-shut...,americaspeaksink.com,2020-08-07 19:45:00,NASA Shuttle Carrier Aircraft Arrives at Kenne...,en,2020,8
108397,SCIENCE,https://www.therahnuma.com/nasa-perseverance-r...,therahnuma.com,2020-08-10 04:52:30,NASA Perseverance rover on way to Mars to find...,en,2020,8


Подготовка пайплайна классификации:

In [25]:
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

model_path = 'tabularisai/multilingual-sentiment-analysis'
sentiment_task = pipeline(
    'sentiment-analysis',
    model=model_path,
    tokenizer=model_path,
    device=device
)

Device set to use cuda:0


In [26]:
text = "I love this product! It's amazing and works perfectly."
sentiment_task(text)

[{'label': 'Very Positive', 'score': 0.5922622084617615}]

Классификация:

In [27]:
examples = df[df.title.str.contains(ORG_NAME)].title.tolist()
examples[:10]

['NASA Releases In-Depth Map of Beirut Explosion Damage',
 'SpaceX, NASA Demo-2 Rocket Launch Set for Saturday: How to Watch',
 'NASA invites engineering students to help harvest water on Mars, Moon',
 "NASA Finds Ammonia-Linked 'Mushballs' and 'Shallow Lightning' on Jupiter",
 '‘It came alive:’ NASA astronauts describe experiencing splashdown in SpaceX Dragon',
 'NASA drops "insensitive" nicknames for cosmic objects',
 'NASA’s InSight lander shows what’s beneath Mars’ surface',
 'SpaceX Crew-1 mission with NASA, first fully operational crewed mission to space to launch in October',
 'ATLAS Space Operations Awarded NASA SBIR Phase II Award to Advance Satellite Constellation Management Scheduling',
 'NASA Astronauts in SpaceX Capsule Splashdown in Gulf Of Mexico']

In [28]:
result = sentiment_task(examples, batch_size=10)
result[:10]

[{'label': 'Very Negative', 'score': 0.478000670671463},
 {'label': 'Neutral', 'score': 0.313614159822464},
 {'label': 'Positive', 'score': 0.38947415351867676},
 {'label': 'Neutral', 'score': 0.2900258004665375},
 {'label': 'Neutral', 'score': 0.3115333020687103},
 {'label': 'Negative', 'score': 0.3855946362018585},
 {'label': 'Neutral', 'score': 0.36988380551338196},
 {'label': 'Very Positive', 'score': 0.4011678695678711},
 {'label': 'Very Positive', 'score': 0.618162214756012},
 {'label': 'Very Negative', 'score': 0.41554921865463257}]

Обработка полученных результатов:

In [29]:
df = pd.DataFrame(
    {
        'title': examples,
        'result': result
    }
)

In [30]:
df[['label', 'score']] = df.result.apply(pd.Series)
df['score'] = np.round(df['score'], 3)
df.drop('result', axis=1, inplace=True)
df

,title,label,score
0,NASA Releases In-Depth Map of Beirut Explosion...,Very Negative,0.478
1,"SpaceX, NASA Demo-2 Rocket Launch Set for Satu...",Neutral,0.314
2,NASA invites engineering students to help harv...,Positive,0.389
3,NASA Finds Ammonia-Linked 'Mushballs' and 'Sha...,Neutral,0.290
4,‘It came alive:’ NASA astronauts describe expe...,Neutral,0.312
...,...,...,...
499,To start with-at any time place helicopter is ...,Neutral,0.442
500,Arecibo Observatory Returns from Tropical Stor...,Neutral,0.384
501,NASA Shuttle Carrier Aircraft Arrives at Kenne...,Neutral,0.312
502,NASA Perseverance rover on way to Mars to find...,Very Positive,0.401


Распределение меток:

In [31]:
new_df = df.label.value_counts().reset_index()
new_df['percentage'] = np.round(new_df['count'] / np.sum(new_df['count']), 2)
new_df

,label,count,percentage
0,Neutral,185,0.37
1,Very Positive,169,0.34
2,Very Negative,60,0.12
3,Positive,50,0.10
4,Negative,40,0.08


Посмотрим примеры по классам:

In [32]:
def show_results(*args: str) -> pd.DataFrame:
    option = df.label.isin(args)

    return df[option]

In [33]:
show_results('Negative', 'Very Negative')

,title,label,score
0,NASA Releases In-Depth Map of Beirut Explosion...,Very Negative,0.478
5,"NASA drops ""insensitive"" nicknames for cosmic ...",Negative,0.386
9,NASA Astronauts in SpaceX Capsule Splashdown i...,Very Negative,0.416
13,‘Like an animal’: NASA astronauts describe noi...,Very Negative,0.472
26,Outrage after NASA ‘goes woke’ and renames ‘in...,Negative,0.330
...,...,...,...
479,Did NASA finally save its Mars mole?,Negative,0.312
481,NASA maps Beirut blast damage through satellit...,Very Negative,0.422
484,Moon landing faked? Astronomer exposes truth b...,Very Negative,0.385
497,NASA Gets Woke by Changing Racist Star Names a...,Very Negative,0.364


In [34]:
show_results('Neutral')


,title,label,score
1,"SpaceX, NASA Demo-2 Rocket Launch Set for Satu...",Neutral,0.314
3,NASA Finds Ammonia-Linked 'Mushballs' and 'Sha...,Neutral,0.290
4,‘It came alive:’ NASA astronauts describe expe...,Neutral,0.312
6,NASA’s InSight lander shows what’s beneath Mar...,Neutral,0.370
15,NASA Administrator Statement on the Passing of...,Neutral,0.357
...,...,...,...
494,"NASA satellite finds 66 new exoplanets, 2,100 ...",Neutral,0.613
499,To start with-at any time place helicopter is ...,Neutral,0.442
500,Arecibo Observatory Returns from Tropical Stor...,Neutral,0.384
501,NASA Shuttle Carrier Aircraft Arrives at Kenne...,Neutral,0.312


In [35]:
show_results('Positive', 'Very Positive')


,title,label,score
2,NASA invites engineering students to help harv...,Positive,0.389
7,"SpaceX Crew-1 mission with NASA, first fully o...",Very Positive,0.401
8,ATLAS Space Operations Awarded NASA SBIR Phase...,Very Positive,0.618
10,Beck teams up with NASA and AI for 'Hyperspace...,Positive,0.438
11,SpaceX brings NASA astronauts safely home,Positive,0.445
...,...,...,...
490,NASA’s OSIRIS-REx Matchpoint Rehearsal: Final ...,Very Positive,0.293
493,NASA satellite images capture stunning video o...,Very Positive,0.471
495,NASA’s Ingenuity Mars helicopter gets first bo...,Very Positive,0.439
496,NASA's planet hunter completes its primary mis...,Very Positive,0.327


# Conclusion:

Заголовки, тексты статей, социальные сети и другие источники информации, содержащие название или связанные с интересующей нас организацией, можно классифицировать для определения позитивных/негативных упомянаний компании, мониторить их появление и соотношение и, основываясь на имеющейся информации, оптимизировать бюджет на маркетинг, а также в срочном порядке реагировать на кейсы, которые могут негативным образом повлиять на репутацию компании